In [0]:
# install dependencies
! pip3 install -r requirements.txt
! git clone https://github.com/mozilla/DeepSpeech.git
! wget https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.tar.gz
! tar -xzf deepspeech-0.7.0-models.tar.gz
! rm deepspeech-0.7.0-models.tar.gz
! pip3 install --upgrade --force-reinstall tensorflow-gpu

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Cloning into 'DeepSpeech'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 18150 (delta 11), reused 16 (delta 7), pack-reused 18117
Receiving objects: 100% (18150/18150), 47.56 MiB | 22.86 MiB/s, done.
Resolving deltas: 100% (12341/12341), done.
--2020-05-04 23:17:45--  https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.tar.gz
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-05-04 23:17:45 ERROR 404: Not Found.

tar (child): deepspeech-0.7.0-models.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
rm: cannot r

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("./drive/My Drive/11785FinalProject")
! pip3 install -r requirements.txt


     |████████████████████████████████| 40kB 4.8MB/s 
  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=1873a7fd8f5e35117c7ed478c1e97f2f78378eec93cbda55892445741c5e5d34
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
path = '/content/drive/My Drive/11785FinalProject/'

In [0]:
import os
os.chdir("/content/drive/My Drive/11785FinalProject")

In [0]:
%pwd

'/content/drive/My Drive/11785FinalProject'

In [0]:
pip install deepspeech

     |████████████████████████████████| 9.7MB 8.0MB/s 


In [0]:
import DeepSpeech

In [0]:
%DeepSpeech -h


UsageError: Line magic function `%DeepSpeech` not found.


In [0]:
## tf_logits.py -- end-to-end differentable text-to-speech
##
## Copyright (C) 2017, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.


import numpy as np
import tensorflow as tf
import argparse

import scipy.io.wavfile as wav

import time
import os
import sys

sys.path.append("DeepSpeech")
import DeepSpeech
from util.text import ctc_label_dense_to_sparse
#from util.flags import create_flags(),FLAG
from util.flags import *


def compute_mfcc(audio, **kwargs):
    """
    Compute the MFCC for a given audio waveform. This is
    identical to how DeepSpeech does it, but does it all in
    TensorFlow so that we can differentiate through it.
    """

    batch_size, size = audio.get_shape().as_list()
    audio = tf.cast(audio, tf.float32)

    # 1. Pre-emphasizer, a high-pass filter
    audio = tf.concat((audio[:, :1], audio[:, 1:] - 0.97*audio[:, :-1], np.zeros((batch_size,512),dtype=np.float32)), 1)

    # 2. windowing into frames of 512 samples, overlapping
    windowed = tf.stack([audio[:, i:i+512] for i in range(0,size-320,320)],1)

    window = np.hamming(512)
    windowed = windowed * window

    # 3. Take the FFT to convert to frequency space
    ffted = tf.compat.v1.spectral.rfft(windowed, [512])
    ffted = 1.0 / 512 * tf.square(tf.abs(ffted))

    # 4. Compute the Mel windowing of the FFT
    energy = tf.reduce_sum(ffted,axis=2)+np.finfo(float).eps
    filters = np.load("filterbanks.npy").T
    feat = tf.matmul(ffted, np.array([filters]*batch_size,dtype=np.float32))+np.finfo(float).eps

    # 5. Take the DCT again, because why not
    feat = tf.compat.v1.log(feat)
    feat = tf.compat.v1.spectral.dct(feat, type=2, norm='ortho')[:,:,:26]

    # 6. Amplify high frequencies for some reason
    _,nframes,ncoeff = feat.get_shape().as_list()
    n = np.arange(ncoeff)
    lift = 1 + (22/2.)*np.sin(np.pi*n/22)
    feat = lift*feat
    width = feat.get_shape().as_list()[1]


    # 7. And now stick the energy next to the features
    feat = tf.concat((tf.reshape(tf.compat.v1.log(energy),(-1,width,1)), feat[:, :, 1:]), axis=2)

    return feat


def get_logits(new_input, length, first=[]):
    """
    Compute the logits for a given waveform.

    First, preprocess with the TF version of MFC above,
    and then call DeepSpeech on the features.
    """

    batch_size = new_input.get_shape()[0]

    # 1. Compute the MFCCs for the input audio
    # (this is differentable with our implementation above)
    empty_context = np.zeros((batch_size, 9, 26), dtype=np.float32)
    new_input_to_mfcc = compute_mfcc(new_input)
    features = tf.concat((empty_context, new_input_to_mfcc, empty_context), 1)

    # 2. We get to see 9 frames at a time to make our decision,
    # so concatenate them together.
    features = tf.reshape(features, [new_input.get_shape()[0], -1])
    features = tf.stack([features[:, i:i+19*26] for i in range(0,features.shape[1]-19*26+1,26)],1)
    features = tf.reshape(features, [batch_size, -1, 19, 26])


    # 3. Finally we process it with DeepSpeech
    # We need to init DeepSpeech the first time we're called
    if first == []:
        first.append(False)

        DeepSpeech.create_flags()
        tf.app.flags.FLAGS.alphabet_config_path = "./DeepSpeech/data/alphabet.txt"
        DeepSpeech.initialize_globals()

    logits, _ = DeepSpeech.BiRNN(features, length, [0]*10)

    return logits

In [0]:
import scipy.io.wavfile as wav

import argparse
from util import *
import tensorflow as tf 
import numpy as np
import os
import sys
from sklearn.metrics import roc_curve, auc
from DeepSpeech import *
# from tf_logits3 import get_logits
from loss import CER, newCER, WER, lcp
sys.path.append("DeepSpeech")




tf.load_op_library = lambda x: x
tmp = os.path.exists
os.path.exists = lambda x: True
import DeepSpeech
os.path.exists = tmp
#os.chdir('DeepSpeech')
print(os.getcwd())
from util.text import ctc_label_dense_to_sparse
# from tf_logits3 import get_logits
from tqdm import tqdm
toks = " abcdefghijklmnopqrstuvwxyz'-"

#DeepSpeech.TrainingCoordinator.__init__ = lambda x: None

#DeepSpeech.TrainingCoordinator.start = lambda x: None

# Use the same decode framework as carlini used :) 
class Decoder:
	def __init__(self, sess, max_audio_len):
		self.sess = sess
		self.max_audio_len = max_audio_len
		self.original = original = tf.Variable(np.zeros((1, max_audio_len), dtype=np.float32), name='qq_original')
		self.lengths = lengths = tf.Variable(np.zeros(1, dtype=np.int32), name='qq_lengths')

		with tf.compat.v1.variable_scope("", reuse=tf.compat.v1.AUTO_REUSE):#tf.variable_scope("", reuse=tf.AUTO_REUSE):
			logits, features = get_logits(original, lengths)

		self.logits = logits
		self.features = features
		saver = tf.train.Saver([x for x in tf.global_variables() if 'qq' not in x.name])
		saver.restore(sess, "models/session_dump")
		self.decoded, _ = tf.nn.ctc_beam_search_decoder(logits, lengths, merge_repeated=False, beam_width=1000)

	def transcribe(self, audio, lengths):
		sess = self.sess
		sess.run(self.original.assign(np.array(audio)))
		sess.run(self.lengths.assign((np.array(lengths)-1)//320))
		out, logits = sess.run((self.decoded, self.logits))
		chars = out[0].values
		res = np.zeros(out[0].dense_shape)+len(toks)-1		
		for ii in range(len(out[0].values)):
			x,y = out[0].indices[ii]
			res[x,y] = out[0].values[ii]
		res = ["".join(toks[int(x)] for x in y).replace("-","") for y in res]
		return res[0]

def decode(audio, num):
	global maxlen
	global D
	global ref
	audios = [list(audio)]
	lengths = [int(maxlen * num)]
	audios = np.array(audios)
	res = D.transcribe(audios, lengths)
	return res

num_samples = 50
y_test = np.zeros(num_samples * 2)
roc_auc = np.zeros(3)
TD = np.zeros((3, num_samples * 2), dtype = np.float32)
count = 0
if __name__ == '__main__':
	sess = tf.compat.v1.Session()#tf.Session()
	parser = argparse.ArgumentParser(description = None)
	#parser.add_argument('--cut', nargs='?', const= 0.5, default=0.5, type = float)
	#args = parser.parse_args()


	ratio = 0.5
	pbar = tqdm(range(num_samples), unit='steps', ascii = True)
	for epoch in pbar:
    # read original wav file
		x, y = wav.read("./DeepSpeech/sample-015110.wav")
    # read adv wav file
		z, w = wav.read("./DeepSpeech/adv-long2long-015110.wav")
		maxlen = len(y)

		#ratio = np.random.random_sample() * 0.6 + 0.2 
		#ratio = (numcut) * 1.0 / (numcut - 1)
		D = Decoder(sess, maxlen)
		stry = decode(y, 1)
		strw = decode(w, 1)
		halfy = decode(y, ratio)
		halfw = decode(w, ratio)

		#print ("Origin: " + stry)
		#print ("Half of Osrigin: " + halfy)
		s1 = newWER(stry, halfy)
		s2 = newCER(stry, halfy)
		s3 = lcp(stry, halfy)

		y_test[count] = 0
		TD[0][count] = float(s1)
		TD[1][count] = float(s2)
		TD[2][count] = float(s3)

		count += 1
		#print ("WER: " + str(s1) + " CER: " + str(s2) + " LCP: " + str(s3))
		#print ("Adv: " + strw)
		#print ("Half of Adv: " + halfw)
		s1 = newWER(strw, halfw)
		s2 = newCER(strw, halfw)
		s3 = lcp(strw, halfw)
		
		y_test[count] = 1
		TD[0][count] = float(s1)
		TD[1][count] = float(s2)
		TD[2][count] = float(s3)
		count += 1
		#print ("WER: " + str(s1) + " CER: " + str(s2) + " LCP: " + str(s3))


	for i in range(3):
		if (i == 2):
			y_test = 1 - y_test
		fpr, tpr, threshold = roc_curve(y_test, TD[i])
		roc_auc[i] = auc(fpr, tpr)

	print ("WER: " + str(roc_auc[0]) + " CER: " + str(roc_auc[1]) + " LCP: " + str(roc_auc[2]))









  0%|          | 0/50 [00:00<?, ?steps/s]

/content/drive/My Drive/11785FinalProject


AttributeError: ignored

In [0]:
%tb

AttributeError: ignored